In [2]:
import pandas as pd
from vega_datasets import data
import altair as alt
import matplotlib.pyplot as plt

In [3]:
alc_df = pd.read_csv('filtered_alc_mort.csv')
alc_df.head()
alc_df["Fatalities"] = alc_df["Data_Value"].replace("-" , 0).fillna(0).astype(int)

In [4]:
gb = alc_df.groupby(['LocationDesc', "Cause_of_Death", "Sex"]).size()
gb

LocationDesc  Cause_of_Death                                      Sex    
Alabama       Air-space transport                                 Female      3
                                                                  Male        3
                                                                  Overall    13
              Alcohol abuse                                       Female      3
                                                                  Male        3
                                                                             ..
Wyoming       Unprovoked seizures, epilepsy, or seizure disorder  Male        3
                                                                  Overall    13
              Water transport                                     Female      3
                                                                  Male        3
                                                                  Overall    13
Length: 9204, dtype: int64

In [5]:
gb = alc_df.groupby(['LocationDesc', 'ConditionType']).count().reset_index()

#counts = gb.size().to_frame(name='counts')

gb = gb.iloc[:,0:3]
gb

,LocationDesc,ConditionType,Unnamed: 0
0,Alabama,Acute,361
1,Alabama,Chronic,822
2,Alabama,Total,19
3,Alaska,Acute,361
4,Alaska,Chronic,822
...,...,...,...
151,Wisconsin,Chronic,822
152,Wisconsin,Total,19
153,Wyoming,Acute,361
154,Wyoming,Chronic,822


In [6]:
counts_p = gb.pivot(index='LocationDesc', columns='ConditionType')
counts_p.head()

Unnamed: 0              
ConditionType      Acute Chronic Total
LocationDesc                          
Alabama              361     822    19
Alaska               361     822    19
Arizona              361     822    19
Arkansas             361     822    19
California           361     822    19

In [7]:
import geopandas
state = geopandas.read_file("ne_110m_admin_1_states_provinces/ne_110m_admin_1_states_provinces.shp")
state.head()

,featurecla,scalerank,adm1_code,diss_me,iso_3166_2,wikipedia,iso_a2,adm0_sr,name,name_alt,...,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,FCLASS_TLC,geometry
0,Admin-1 scale rank,2,USA-3514,3514,US-MN,http://en.wikipedia.org/wiki/Minnesota,US,1,Minnesota,MN|Minn.,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-89.95766 47.28691, -90.13175 47.292..."
1,Admin-1 scale rank,2,USA-3515,3515,US-MT,http://en.wikipedia.org/wiki/Montana,US,1,Montana,MT|Mont.,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-116.04823 49.00037, -113.05950 49.0..."
2,Admin-1 scale rank,2,USA-3516,3516,US-ND,http://en.wikipedia.org/wiki/North_Dakota,US,1,North Dakota,ND|N.D.,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-97.22894 49.00089, -97.21414 48.902..."
3,Admin-1 scale rank,2,USA-3517,3517,US-HI,http://en.wikipedia.org/wiki/Hawaii,US,8,Hawaii,HI|Hawaii,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-155.93665 19.05939, -155.9080..."
4,Admin-1 scale rank,2,USA-3518,3518,US-ID,http://en.wikipedia.org/wiki/Idaho,US,1,Idaho,ID|Idaho,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-116.04823 49.00037, -115.96780 47.9..."


In [8]:
full = pd.merge(state, alc_df, left_on='woe_name', right_on='LocationDesc', how='outer')
full.head()
full.shape

(62504, 139)

In [9]:
states = alt.topo_feature(data.us_10m.url, feature='states')
states

UrlData({
  format: TopoDataFormat({
    feature: 'states',
    type: 'topojson'
  }),
  url: 'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json'
})

In [14]:
alc_df.head()

,Unnamed: 0,LocationDesc,ConditionType,Category,Cause_of_Death,Data_Value,Data_Value_Alt,Data_Value_Footnote_Symbol,Data_Value_Footnote,Effect,ConsumptionPattern,Sex,AgeCategory,AgeGroup,ConditionTypeID,AgeCategoryID,Fatalities
0,1,West Virginia,Chronic,Total,Total for All Causes,0,0.0,NaN,NaN,Harmful Effects,Excessive Alcohol Use,Female,Under 21,Overall,EXCSVE,UN21,0
1,2,West Virginia,Total,Total,Total for All Causes,7,7.0,NaN,NaN,Harmful Effects,Excessive Alcohol Use,Female,Under 21,Overall,EXCSVE,UN21,7
2,3,United States,Acute,Alcohol-Related Poisonings,Alcohol poisoning,13,13.0,NaN,NaN,Harmful Effects,Excessive Alcohol Use,Female,Under 21,Under 21,ACUTE,UN21,13
3,4,United States,Acute,Alcohol-Related Poisonings,Alcohol poisoning,1761,1761.0,NaN,NaN,Harmful Effects,Excessive Alcohol Use,Male,Other,Overall,ACUTE,OTH,1761
4,5,West Virginia,Acute,Total,Total for All Causes,367,367.0,NaN,NaN,Harmful Effects,Any Alcohol Use,Male,Other,Overall,ANYALC,OTH,367


In [11]:
alc_df_age = alc_df.groupby(by = ['LocationDesc', 'AgeGroup'], as_index= False).sum("Fatalities")

alc_df_age = alc_df_age.iloc[:, [0,1,4]]
alc_df_age = alc_df_age[alc_df_age["AgeGroup"] != "Under 21"]
alc_df_age

,LocationDesc,AgeGroup,Fatalities
0,Alabama,0-19,346
1,Alabama,20-34,1984
2,Alabama,35-49,2537
3,Alabama,50-64,3945
4,Alabama,65+,3940
...,...,...,...
358,Wyoming,20-34,206
359,Wyoming,35-49,360
360,Wyoming,50-64,598
361,Wyoming,65+,446


In [12]:
full = pd.merge(state, alc_df_age, left_on='woe_name', right_on='LocationDesc', how='outer')
full.head()
full.shape

(312, 125)

In [13]:
import altair as alt
from vega_datasets import data
import geopandas as gpd

ages = full['AgeGroup'].unique()

input_dropdown = alt.binding_select(options=['Europe','Japan','USA'], name='Region ')
selection = alt.selection_point(fields=['Origin'], bind=input_dropdown)
color = alt.condition(
    selection,
    alt.Color('Origin:N').legend(None),
    alt.value('lightgray')
)

# define a pointer selection
click_state = alt.selection_point(fields=["LocationDesc"])

# create a choropleth map using a lookup transform
# define a condition on the opacity encoding depending on the selection
choropleth = (
    alt.Chart(full)
    .mark_geoshape()
    .transform_lookup(
        lookup="id", from_=alt.LookupData(us_population, "id", ["population", "state"])
    )
    .encode(
        color="population:Q",
        opacity=alt.condition(click_state, alt.value(1), alt.value(0.2)),
        tooltip=["state:N", "population:Q"],
    )
    .project(type="albersUsa")
)

# create a bar chart with a similar condition on the opacity encoding.
bars = (
    alt.Chart(
        us_population.nlargest(15, "population"), title="Top 15 states by population"
    )
    .mark_bar()
    .encode(
        x="population",
        opacity=alt.condition(click_state, alt.value(1), alt.value(0.2)),
        color="population",
        y=alt.Y("state").sort("-x"),
    )
)

(choropleth & bars).add_params(click_state)

NameError: name 'us_population' is not defined

In [ ]:
full['geometry'] = full.geometry

In [ ]:
ages = full['AgeGroup'].unique()
ages = list(set(ages))
ages

In [ ]:
# List of all unique Measures
measures = ages

# Create drop-down, using the "measures" list
input_dropdown = alt.binding_select(options=measures, name='Age Groups')

# Creating a binding "selection" object in Vega
# which is mapped filter the "Measure" field in the data
selection = alt.selection_point(fields=['Age Groups'], bind=input_dropdown)

state_fig = (
    alt.Chart(full)   # Note I removed the .filter()
    .mark_geoshape(strokeWidth=1, stroke="darkgreen")
    .encode(
        shape="geometry:G",
        color="Value",
        tooltip=['LocationDesc', 'AgeGroup', "Fatalities"],
    )
    .transform_lookup(
        lookup="geometry", 
        from_=alt.LookupData(data=counties, key="id"), 
        as_="geometry"
    )
    .transform_filter(   # transform_filter() allows dynamic selection
        selection  
    )
    .add_params(   # add_params() make "selection" variable available to chart
        selection
    )
    .project(type="albersUsa")
    .properties(width=400)
)
state_fig

In [ ]:
counties = alt.topo_feature(data.us_10m.url, 'state')

In [ ]:
import polars as pl
alt.data_transformers.disable_max_rows()

state = alt.topo_feature(data.us_10m.url, 'state')

# List of all unique Measures
measures = ages

# Create drop-down, using the "measures" list
input_dropdown = alt.binding_select(options=measures, name='Measure')

# Creating a binding "selection" object in Vega
# which is mapped filter the "Measure" field in the data
selection = alt.selection_point(fields=['Measure'], bind=input_dropdown)

state_fig = (
    alt.Chart(full.filter("AgeGroup")=="Overall")   # Note I removed the .filter()
    .mark_geoshape(strokeWidth=1, stroke="darkgreen")
    .encode(
        shape="geometry:G",
        color="Fatalities",
        tooltip=['LocationDesc', 'AgeGroup', "Fatalities"]
    ).add_params(   # add_params() make "selection" variable available to chart
        selection
    )
    .project(type="albersUsa")
    .properties(width=400)
)
state_fig

In [ ]:
    ).transform_filter(   # transform_filter() allows dynamic selection
        selection  
    )


state_fig = (
    alt.Chart(full.filter(pl.col("Measure")=="Total Population"))
    .mark_geoshape(strokeWidth=1, stroke="darkgreen")
    .encode(
        shape="geo:G",
        color="Value",
        tooltip=["County Name", "GEOID", "Measure", "Value"],
    )
    .transform_lookup(
        lookup="GEOID", 
        from_=alt.LookupData(data=counties, key="id"), 
        as_="geo"
    )
    .project(type="albersUsa")
    .properties(width=400)
)
state_fig

In [ ]:
import altair as alt
from vega_datasets import data

# Load the US state data
states = alt.topo_feature(data.us_10m.url, 'states')

# Create a dropdown selection
category_dropdown = alt.binding_select(options=['Population', 'Unemployment Rate'])
selection = alt.selection_single(fields=['category'], bind=category_dropdown, name='Select')

# Define the base map
base = alt.Chart(states).mark_geoshape(
    fill='#EEE',
    stroke='white'
).properties(
    width=500,
    height=300
).project('albersUsa')

# Define the choropleth map
choropleth = alt.Chart(states).mark_geoshape().encode(
        color=alt.condition(
            selection,
            alt.Color('properties.category:N', scale=alt.Scale(scheme='blues')),
            alt.value('lightgray')
        ),
        tooltip=[
            alt.Tooltip('properties.name:N', title='State'),
            alt.Tooltip('properties.category:Q', title='Category Value')
        ]
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(data=states, key='id', fields=['category'])
    ).add_params(
        selection
    ).properties(
        title="US Chloropleth Map"
    )

# Combine the base map and the choropleth map
us_choropleth = base + choropleth

# Show the map
us_choropleth

In [ ]:
import pandas as pd
pop_df = pd.read_csv('nst-est2019-01.csv')
pop_df['state'] = list(map(lambda x: x.replace('.', ''), pop_df['state']))